# Carbon Free Boston

In [1]:
import emat
emat.__version__

C:\Users\jnewman\AppData\Local\Continuum\anaconda3\envs\EMAT\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
c:\users\jnewman\git\emaworkbench\ema_workbench\connectors\__init__.py:8: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)
c:\users\jnewman\git\emaworkbench\ema_workbench\connectors\__init__.py:18: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)
c:\users\jnewman\git\emaworkbench\ema_workbench\connectors\__init__.py:23: ImportWarning: pysd connector not available
  warnings.warn("pysd connector not available", ImportWarning)


'0.1.1'

In [2]:
import os

In [3]:
import ema_workbench

from ema_workbench import TimeSeriesOutcome, ema_logging, perform_experiments
from ema_workbench.connectors.excel import ExcelModel
from ema_workbench.em_framework.evaluators import MultiprocessingEvaluator, SequentialEvaluator
from ema_workbench import BooleanParameter, IntegerParameter, RealParameter, ScalarOutcome, Constraint

In [4]:
ema_logging.log_to_stderr(level=ema_logging.INFO)

<Logger EMA (DEBUG)>

In [5]:
excel_file = "./CFB/CFB-Trans_v1.3.3.xlsm"
assert os.path.exists(excel_file)

In [6]:
scope_file = './CFB/CFB-emat.yaml'
assert os.path.exists(scope_file)

In [7]:
import os
wd, model_file = os.path.split(excel_file)
wd, model_file

('./CFB', 'CFB-Trans_v1.3.3.xlsm')

In [8]:
model = emat.ExcelCoreModel(
    wd=wd, 
    model_file=model_file,
    scope=scope_file,
)

## Using a Database

The exploratory modeling process will typically generate many different sets of outputs,
for different explored modeling scopes, or for different applications.  It is convenient
to organize these outputs in a database structure, so they are stored consistently and 
readily available for subsequent analysis.

The `SQLiteDB` object will create a database to store results.  When instantiated with
no arguments, the database is initialized in-memory, which will not store anything to
disk (which is convenient for this example, but in practice you will generally want to
store data to disk so that it can persist after this Python session ends).

In [14]:
db = emat.SQLiteDB('./CFB/CFB3.sqldb', initialize=True)

An EMAT Scope can be stored in the database, to provide needed information about what the 
various inputs and outputs represent.

In [15]:
model.scope.store_scope(db)
model.db = db

Trying to store another scope with the same name (or the same scope) raises a KeyError.

In [16]:
from emat.experiment.experimental_design import design_experiments

In [17]:
design = design_experiments(model.scope, db=db, n_samples=500, sampler='lhs')
design.head()

,% CAV market penetration: LDV,% CAV market penetration: MDT,Active Transportation: Bike growth factor,Active Transportation: E-bike % of bike miles,Active Transportation: New bike facility miles,Active Transportation: Prior drive mode share of new bike & walk trips:,Active Transportation: Share of <2 mi trips converted to walk,Active Transportation: Use growth factor or elasticity?,CAV (Private),CAV - % VMT change: LDV,...,Transit vehicles,Transit,Operational improvements,New service,Active Transportation,Smart Mobility,Ridehailing,Demand response/microtransit:,TDM,constant
experiment,,,,,,,,,,,,,,,,,,,,,
1,13,67,21,23,14,54,10,growth factor,0,89,...,True,True,True,True,True,True,True,True,True,1.0
2,84,91,76,43,57,30,50,growth factor,1,24,...,True,True,True,True,True,True,True,True,True,1.0
3,70,82,55,0,163,98,4,growth factor,1,68,...,True,True,True,True,True,True,True,True,True,1.0
4,72,99,74,54,144,54,3,growth factor,0,55,...,True,True,True,True,True,True,True,True,True,1.0
5,22,33,56,55,16,90,47,elasticity,0,98,...,True,True,True,True,True,True,True,True,True,1.0


In [18]:
with MultiprocessingEvaluator(model, n_processes=4) as evaluator:
    lhs_results = model.run_experiments(design_name='lhs', evaluator=evaluator, db=db)
lhs_results.head()

[Mon Feb 04 16:52:58 MainProcess/INFO] pool started
[Mon Feb 04 16:52:58 MainProcess/INFO] performing 500 scenarios/policies * 1 model(s) = 500 experiments
[Mon Feb 04 16:54:48 MainProcess/INFO] 50 cases completed
[Mon Feb 04 16:56:25 MainProcess/INFO] 100 cases completed
[Mon Feb 04 16:58:05 MainProcess/INFO] 150 cases completed
[Mon Feb 04 16:59:39 MainProcess/INFO] 200 cases completed
[Mon Feb 04 17:01:23 MainProcess/INFO] 250 cases completed
[Mon Feb 04 17:02:57 MainProcess/INFO] 300 cases completed
[Mon Feb 04 17:04:38 MainProcess/INFO] 350 cases completed
[Mon Feb 04 17:06:14 MainProcess/INFO] 400 cases completed
[Mon Feb 04 17:07:57 MainProcess/INFO] 450 cases completed
[Mon Feb 04 17:09:25 MainProcess/INFO] 500 cases completed
[Mon Feb 04 17:09:25 MainProcess/INFO] experiments finished
[Mon Feb 04 17:09:25 MainProcess/INFO] terminating pool


,Grid GHG intensity,Light-duty vehicles: % of LDV stock: EV,Light-duty vehicles: % of LDV stock: PHEV,Trucks: % of MDT stock: EV,Transit vehicles: % of transit bus fleet: EV,Transit vehicles: % of school bus fleet: EV,Transit vehicles: % of commuter rail electrified,Land Use,Prior drive mode share of new riders: bus,Prior drive mode share of new riders: urban rail,...,GHG Change 2030: Combined,GHG Change 2050: Clean Vehicles,GHG Change 2050: Land Use,GHG Change 2050: Transit,GHG Change 2050: Active Transportation,GHG Change 2050: CAV,GHG Change 2050: Smart Mobility,GHG Change 2050: Travel Pricing,GHG Change 2050: TDM,GHG Change 2050: Combined
experiment,,,,,,,,,,,,,,,,,,,,,
1,328,53,46,20,93,53,68,False,37,57,...,8.157580e+05,-3019.615744,0.0,-5501.924944,-53602.354617,0.0,-133541.729707,0.0,0.0,1.002030e+06
2,396,10,35,65,66,63,60,False,26,34,...,1.094071e+06,-3022.876885,0.0,-2975.222974,-36684.320243,0.0,-135121.979234,0.0,0.0,1.017575e+06
3,362,85,96,63,79,3,62,True,79,42,...,8.061697e+06,-3023.963932,0.0,-5588.996588,-84618.750969,0.0,-141435.132026,0.0,0.0,9.673237e+05
4,25,55,23,40,18,54,37,True,37,57,...,6.035218e+05,-3012.006414,0.0,-5501.924944,-53602.354617,0.0,-135114.847339,0.0,0.0,1.000542e+06
5,223,64,44,63,13,64,35,True,88,36,...,7.330112e+05,-3012.821699,0.0,-5348.630919,-78979.406178,0.0,-139849.711885,0.0,0.0,9.739368e+05


In [ ]:
# model.constraints = [

# Constraint(
#     'Light-duty vehicles: Total Fleet Size', 
#     parameter_names=[
#         "Light-duty vehicles: % of LDV stock: EV",
#         "Light-duty vehicles: % of LDV stock: PHEV",
#     ],
#     function= lambda *x: 100-sum(*x),
# )
# ]

In [ ]:
# model.get_feature_scores('lhs')

In [ ]:
model.db = db

In [19]:
lhs_results.to_csv('./CFB/CFB-500.csv')

In [20]:
db

In [25]:
db.read_experiment_all('CFB','lhs')

,Clean Vehicles,Light-duty vehicles,Trucks,Transit vehicles,Transit,Operational improvements,New service,Active Transportation,Smart Mobility,Ridehailing,...,GHG Change 2030: Combined,GHG Change 2050: Clean Vehicles,GHG Change 2050: Land Use,GHG Change 2050: Transit,GHG Change 2050: Active Transportation,GHG Change 2050: CAV,GHG Change 2050: Smart Mobility,GHG Change 2050: Travel Pricing,GHG Change 2050: TDM,GHG Change 2050: Combined
experiment,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,1,1,1,1,1,1,...,815758,-3019.62,0,-5501.92,-53602.4,0,-133542,0,0,1.00203e+06
2,1,1,1,1,1,1,1,1,1,1,...,1.09407e+06,-3022.88,0,-2975.22,-36684.3,0,-135122,0,0,1.01758e+06
3,1,1,1,1,1,1,1,1,1,1,...,8.0617e+06,-3023.96,0,-5589,-84618.8,0,-141435,0,0,967324
4,1,1,1,1,1,1,1,1,1,1,...,603522,-3012.01,0,-5501.92,-53602.4,0,-135115,0,0,1.00054e+06
5,1,1,1,1,1,1,1,1,1,1,...,733011,-3012.82,0,-5348.63,-78979.4,0,-139850,0,0,973937
6,1,1,1,1,1,1,1,1,1,1,...,830251,-3016.08,0,-8785.7,-26110.5,0,-139852,0,0,1.01729e+06
7,1,1,1,1,1,1,1,1,1,1,...,25142.1,-3015.81,0,-8111.7,-26110.5,0,-141430,0,0,1.01634e+06
8,1,1,1,1,1,1,1,1,1,1,...,563190,-3023.69,0,-5472.9,-15536.8,0,-144591,0,0,1.02474e+06
9,1,1,1,1,1,1,1,1,1,1,...,568101,-3013.64,0,-7740.12,-76159.7,0,-143006,0,0,971501


In [22]:
db.read_scope_names()

['CFB']

In [24]:
db.read_design_names('CFB')

['lhs']

In [26]:
db.close()

AttributeError: 'SQLiteDB' object has no attribute 'close'

In [27]:
db.conn.close()